# Train model and do test

In [29]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from preprocess_data import *
from Hyperparameter_optimization import *
from Degree_selection import *
from proj1_helpers import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from proj1_helpers import *

## Load the training set
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y_train, tX_train, ids_train = load_csv_data(DATA_TRAIN_PATH)
feature_names = load_headers(DATA_TRAIN_PATH)

# convert y from -1/1 to 0,1
y_train = (y_train+1)/2

# process train set
data_pro  = data_preprocess_train(tX_train,y_train,outlier_method = 'IQR',n_out = 5)

## Load the test set

# Load test set
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
feature_names = load_headers(DATA_TEST_PATH)
# process test set
data_test  = data_preprocess_predict(tX_test,outlier_method = 'IQR',n_out = 5)

# Train a model and see accuracy

In [43]:
lambda_ = [1e-3]
#lambda_ = np.linspace(1e-3,1e-5,4)
gamma_ = [0] # np.logspace(-4,-1,4) 
accuracy_ = []
group_size_ = []
optimal_lambdas = []
degrees = [10, 10, 10, 4, 10, 5]
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    tx = np.hstack((build_poly(tx, degrees[idx_data]),np.sin(tx)))
    y = data[1]. reshape(-1,1)
    initial_w_ = np.zeros((tx.shape[1],1))
    optimal_accuracy, optimal_lambda_, optimal_gamma = Hyperparameter_optimization( y, tx,k_fold=5, 
                                                                                   lambdas=lambda_ , 
                                                                                   gammas = gamma_, 
                                                                                   initial_w = initial_w_, 
                                                                                   max_iters = 0, 
                                                                                   model = 'ridge_regression')
                                                                                   #model = 'least_squares')
    accuracy_.append(optimal_accuracy)
    group_size_.append(len(data[2]))
    optimal_lambdas.append(optimal_lambda_)

accuracy_all = np.dot(accuracy_,group_size_)/np.sum(group_size_)
print('\n * Overall Accuracy = %.5f'%accuracy_all)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0.001,  gamma: 0 
 
 Model ridge_regression 
 Loss: 0.0691764300749895 
 Accuracy: 0.8075891042146633
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: ridge_regression 
 Optimal accuracy: 0.8075891042146633 
 Optimal lambda_: 0.001 
 Optimal gamma: 0 
 
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0.001,  gamma: 0 
 
 Model ridge_regression 
 Loss: 0.020952743676925917 
 Accuracy: 0.9480857580398162
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: ridge_regression 
 Optimal accuracy: 0.9480857580398162 
 Optimal lambda_: 0.001 
 Optimal gamma: 0 
 
┌──────────────────────────────────────┐
│               Group: 2               │


In [ ]:
optimal_lambdas

# Train model and do prediction on test set

In [10]:
#from preprocess_data import *

#data_pro  = data_preprocess_train(tX_train,y_train,outlier_method = 'IQR',n_out = 5)

# Train
lambda_ = [6.5e-05,
 1e-05,
 0.0009,
 0.000835,
 0.00078,
 0.000615]
loss_ = [] 
w_ = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    tx = np.hstack((build_poly(tx, degrees[idx_data]),1/(abs(tx)+1e-10),np.sin(tx)))
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    loss_temp, w_temp = ridge_regression(y, tx,lambda_[idx_data])
    w_.append(w_temp)
    print('The training of Group {} is over.'.format(idx_data))
#print('\n * Overall Accuracy = %.4f'%accuracy_all)

print('\nTraining finished! Start do prediction and write to prediction.csv.')

# Using trained model to do prediction
label_ = []
data_size = sum([len(data_test[:,1][i]) for i in range(6)])
y_predict = np.zeros(data_size)
for idx_data,data in enumerate(data_test):
    tx = data[0]
    tx = np.hstack((build_poly(tx, degrees[idx_data]),1/(abs(tx)+1e-10),np.sin(tx)))
    label_.append(predict_labels(w_[idx_data], tx,model='ridge_regression'))
    y_predict[data[1]] = predict_labels(w_[idx_data], tx,model='ridge_regression')

create_csv_submission(range(350000,350000+data_size),(y_predict-0.5)*2,'prediction.csv')

print('\nPrediction finished! Please check corresponding prediction.csv in ../scripts/.')

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
The training of Group 0 is over.
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
The training of Group 1 is over.
┌──────────────────────────────────────┐
│               Group: 2               │
└──────────────────────────────────────┘
The training of Group 2 is over.
┌──────────────────────────────────────┐
│               Group: 3               │
└──────────────────────────────────────┘
The training of Group 3 is over.
┌──────────────────────────────────────┐
│               Group: 4               │
└──────────────────────────────────────┘
The training of Group 4 is over.
┌──────────────────────────────────────┐
│               Group: 5               │
└──────────────────────────────────────┘
The training of Group 5 is over.

Training finished! Start do prediction and write to prediction.